## Run this notebook

You can launch this notbook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=example-notebooks%2Fwfs.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Use `OWSLib` to determine what data is available and inspect the metadata
   2. Use `OWSLib` to filter and read the data
   3. Use `geopandas` and `folium` to analyze and plot the data

Note that the default examples environment is missing one requirement: `oswlib`. We can `pip install` that before we move on. 

In [1]:
!pip install OWSLib==0.28.1

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from owslib.ogcapi.features import Features
import geopandas as gpd
import datetime as dt
from datetime import datetime, timedelta

## Look at the data that is availible through the OGC API

The datasets that are distributed throught the OGC API are organized into collections.
We can display the collections with the command: 

In [3]:
OGC_URL = "https://firenrt.delta-backend.com"

w = Features(url=OGC_URL)
w.feature_collections()

['public.eis_fire_newfirepix',
 'public.eis_fire_fireline',
 'public.eis_fire_perimeter',
 'public.st_subdivide',
 'public.st_hexagongrid',
 'public.st_squaregrid']

We will focus on the `public.eis_fire_fireline` collection and the `public.eis_fire_perimeter` collection. 

### Inspect the metatdata for public.eis_fire_perimeter collection

We can access information that drescribes the `public.eis_fire_perimeter`. 

In [4]:
perm = w.collection("public.eis_fire_perimeter")

We are particularly interested in the spatial and temporal extents of the data.

In [5]:
perm["extent"]

{'spatial': {'bbox': [[-124.5470962524414,
    24.073780059814453,
    -62.94002914428711,
    49.39276123046875]],
  'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'},
 'temporal': {'interval': [['2023-03-07T00:00:00+00:00',
    '2023-03-27T00:00:00+00:00']],
  'trs': 'http://www.opengis.net/def/uom/ISO-8601/0/Gregorian'}}

In addition to getting metadata about the data we can access the queryable fields. Each of these fields will represent a column in our dataframe.

In [6]:
perm_q = w.collection_queryables("public.eis_fire_perimeter")
perm_q["properties"]

{'wkb_geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'ogc_fid': {'name': 'ogc_fid', 'type': 'number'},
 'n_pixels': {'name': 'n_pixels', 'type': 'number'},
 'n_newpixels': {'name': 'n_newpixels', 'type': 'number'},
 'farea': {'name': 'farea', 'type': 'number'},
 'fperim': {'name': 'fperim', 'type': 'number'},
 'flinelen': {'name': 'flinelen', 'type': 'number'},
 'duration': {'name': 'duration', 'type': 'number'},
 'pixden': {'name': 'pixden', 'type': 'number'},
 'meanfrp': {'name': 'meanfrp', 'type': 'number'},
 'isactive': {'name': 'isactive', 'type': 'number'},
 't': {'name': 't', 'type': 'string'},
 'fireid': {'name': 'fireid', 'type': 'number'}}

## Filter the data

It is always a good idea to do any data filtering as early as possible. In this example we know that we want the data for particular spatial and temporal extents. We can apply those and other filters using the `OWSLib` package.

In the below example we are:

 - choosing the `public.eis_fire_perimeter` collection 
 - subsetting it by space using the `bbox` parameter
 - subsetting it by time using the `datetime` parameter
 - filtering for fires over 5km^2 and over 2 days long using the `filter` parameter. The `filter` parameter lets us filter by the columns in 'public.eis_fire_perimeter' using SQL-style queries. 

NOTE: The `limit` parameter desginates the maximum number of objects the query will return. The default limit is 10, so if we want to all of the fire perimeters within certain conditions, we need to make sure that the limit is large.

In [7]:
## Get the most recent fire perimeters, and 7 days before most recent fire perimeter
most_recent_time = max(*perm["extent"]["temporal"]["interval"])
now = dt.datetime.strptime(most_recent_time, "%Y-%m-%dT%H:%M:%S+00:00")
last_week = now - dt.timedelta(weeks=1)
last_week = dt.datetime.strftime(last_week, "%Y-%m-%dT%H:%M:%S+00:00")
print("Most Recent Time =", most_recent_time)
print("Last week =", last_week)

Most Recent Time = 2023-03-27T00:00:00+00:00
Last week = 2023-03-20T00:00:00+00:00


In [8]:
perm_results = w.collection_items(
    "public.eis_fire_perimeter",  # name of the dataset we want
    bbox=["-106.8", "24.5", "-72.9", "37.3"],  # coodrinates of bounding box,
    datetime=[last_week + "/" + most_recent_time],  # date range
    limit=1000,  # max number of items returned
    filter="farea>5 AND duration>2",  # additional filters based on queryable fields
)

The result is a dictionary containing all of the data and some summary fields. We can look at the keys to see what all is in there.

In [9]:
perm_results.keys()

dict_keys(['type', 'id', 'title', 'description', 'numberMatched', 'numberReturned', 'links', 'features'])

For instance you can check the total number of matched items and make sure that it is equal to the number of returned items. This is how you know that the `limit` you defined above is high enough.

In [10]:
perm_results["numberMatched"] == perm_results["numberReturned"]

True

You can also access the data directly in the browser or in an HTTP GET call using the constructed link.

In [11]:
perm_results["links"][1]["href"]

'https://firenrt.delta-backend.com/collections/public.eis_fire_perimeter/items?bbox=-106.8%2C24.5%2C-72.9%2C37.3&datetime=2023-03-20T00%3A00%3A00%2B00%3A00%2F2023-03-27T00%3A00%3A00%2B00%3A00&limit=1000&filter=farea%3E5+AND+duration%3E2'

## Read data

In addition to all the summary fields, the `perm_results` dict contains all the data. We can pass the data into geopandas to make it easier to interact with.

In [12]:
df = gpd.GeoDataFrame.from_features(perm_results["features"])
df

geometry  ogc_fid  n_pixels  \
0   POLYGON ((-81.10837 28.22902, -81.10836 28.229...      100        37   
1   POLYGON ((-95.65553 30.53358, -95.65626 30.536...      621        37   
2   POLYGON ((-84.85822 30.25672, -84.85808 30.256...      957       200   
3   POLYGON ((-84.11511 30.59867, -84.11521 30.598...     1009       228   
4   POLYGON ((-83.93334 30.39572, -83.93331 30.395...     1021        63   
..                                                ...      ...       ...   
60  MULTIPOLYGON (((-81.21954 32.61315, -81.22386 ...     5839        41   
61  MULTIPOLYGON (((-77.87873 37.01233, -77.87890 ...     6330       130   
62  MULTIPOLYGON (((-103.18058 31.26391, -103.1805...     9516        48   
63  POLYGON ((-100.05205 28.42725, -100.05202 28.4...     9833       102   
64  POLYGON ((-99.64565 28.32640, -99.64570 28.326...     9839         4   

    n_newpixels      farea     fperim  flinelen  duration     pixden  meanfrp  \
0             0   7.983032  13.747705       0.0       4.0   4.634831      0.0   
1             0   5.385105   9.209855       0.0       4.0   6.870804      0.0   
2             0  47.280059  46.224696       0.0       9.0   4.230113      0.0   
3             0  57.264856  42.051987       0.0      21.5   3.981500      0.0   
4             0  19.526074  38.203165       0.0      10.0   3.226455      0.0   
..          ...        ...        ...       ...       ...        ...      ...   
60            0   8.634942  23.111709       0.0      16.0   4.748150      0.0   
61            0  21.360419  27.420088       0.0       7.0   6.086023      0.0   
62            0  16.331298  34.518061       0.0      30.0   2.939142      0.0   
63            0   5.967796  14.815321       0.0      15.0  17.091738      0.0   
64            0   9.627206  14.289996       0.0       3.0   0.415489      0.0   

    isactive                    t  fireid  
0          1  2023-03-24T12:00:00   34008  
1          1  2023-03-23T12:00:00   33468  
2          1  2023-03-24T12:00:00   30924  
3          1  2023-03-24T12:00:00   23632  
4          1  2023-03-24T12:00:00   30028  
..       ...                  ...     ...  
60         1  2023-03-24T12:00:00   27881  
61         0  2023-03-21T12:00:00   30377  
62         1  2023-03-24T00:00:00   19555  
63         1  2023-03-26T12:00:00   29087  
64         1  2023-03-25T00:00:00   35385  

[65 rows x 13 columns]

## Explore data

We can quickly explore the data by setting the coordinate reference system (`crs`) and using `.explore()`

In [13]:
df = df.set_crs("EPSG:4326")
df.explore()

## Visualize Most Recent Fire Perimeters with Firelines

If we wanted to combine collections to make more informative analyses, we can use some of the same principles. 

First we'll get the queryable fields, and the extents:

In [14]:
fline_q = w.collection_queryables("public.eis_fire_fireline")
fline_collection = w.collection("public.eis_fire_fireline")
fline_q["properties"]

{'wkb_geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'ogc_fid': {'name': 'ogc_fid', 'type': 'number'},
 'fireid': {'name': 'fireid', 'type': 'number'},
 't': {'name': 't', 'type': 'string'},
 'mergeid': {'name': 'mergeid', 'type': 'number'}}

### Read
Then we'll use those fields to get most recent fire perimeters and fire lines.

In [15]:
perm_results = w.collection_items(
    "public.eis_fire_perimeter",
    datetime=most_recent_time,
    limit=1000,
)
perimeters = gpd.GeoDataFrame.from_features(perm_results["features"])

## Get the most recent fire lines
perimeter_ids = perimeters.fireid.unique()
perimeter_ids = ",".join(map(str, perimeter_ids))

fline_results = w.collection_items(
    "public.eis_fire_fireline",
    limit=1000,
    filter="fireid IN ("
    + perimeter_ids
    + ")",  # only the fires from the fire perimeter query above
)
fline = gpd.GeoDataFrame.from_features(fline_results["features"])

### Visualize

In [16]:
perimeters = perimeters.set_crs("epsg:4326")
fline = fline.set_crs("epsg:4326")

m = perimeters.explore()
m = fline.explore(m=m, color="orange")
m

## Download Data

Downloading pre-filtered data may be useful for working locally, or for working with the data in GIS software. 


We can download the dataframe we made by writing it out into a shapefile or into a GeoJSON file.  

    perimeters.to_file('perimeters.shp') 
    perimeters.to_file('perimeters.geojson', driver='GeoJSON')

## About the Data


The fire data shown is generated by the FEDs algorithm. The FEDs algorithm tracks fire movement and severity by ingesting observations from the VIIRS thermal sensors on the Suomi NPP and NOAA-20 satellites. This algorithm uses raw VIIRS observations to generate a polygon of the fire, locations of the active fire line, and estimates of fire mean Fire Radiative Power (FRP). The VIIRS sensors overpass at ~1:30 AM and PM local time, and provide estimates of fire evolution ~ every 12 hours. The data produced by this algorithm describe where fires are in space and how fires evolve through time. This CONUS-wide implementation of the FEDs algorithm is based on [Chen et al 2020’s](https://www.nature.com/articles/s41597-022-01343-0) algorithm for California.


The data produced by this algorithm is considered experimental. 


## Collection Information

**public.eis_fire_perimeter**

Perimeter of cumulative fire-area.

| Column      | Description | Unit      |
| ----------- | ----------- | ----------- |
| meanfrp      | Mean fire radiative power. The weighted sum of the fire radiative power detected at each new pixel, divided by the number of pixels.   If no new pixels are detected, meanfrp is set to zero. | MW/(pixel area)  |
| t   | Time of most recent VIIRS detection. API delivers all perimeters with a VIIRS detection in the last 20 days.    | Datetime. yyyy-mm-ddThh:mm:ss. Local time.  |
| fireid   | Fire ID. Unique for each fire. Matches fireid. |Numeric ID |
| pixden   |Number of pixels divided by area of perimeter.   | pixels/Km^2|
| duration  |Number of days since first observation of fire. Fires with a single observation have a duration of zero. |Days|
| flinelen   | Length of active fire line, based on new pixels. If no new pixels are detected, flinelen is set to zero.   |Km |
| fperim   | Length of fire perimeter.        | Km   |
| farea   | Area within fire perimeter.        |Km^2    |
| n_newpixels   | Number of pixels newly detected since last overpass.      |pixels  |
| n_pixels   | Number of pixel-detections in history of fire. | pixels   |
| isactive   | Have new fire pixels been detected in the last 5 days?     | Boolean |
| ogc_fid   |The ID used by the OGC API to sort perimeters.       | Numeric ID   |
| geometry  |The shape of the perimeter.    | Geometry    |


**public.eis_fire_fireline**

Active fire line as estimated by new VIIRS detections.

**public.eis_fire_newfirepix**

New pixel detections that inform the most recent time-step’s perimeter and fireline calculation.

| Column      | Description | Unit      |
| ----------- | ----------- | ----------- |
| fireid   | ID of fire pixel associated with.      | Numeric ID |
| t   | Most recent algorithm execution.  Note: This is not time of detection. The time of detection for newfirepix and fireline is the 't' of the perimeter with a matching ID. | Datetime. yyyy-mm-ddThh:mm:ss.|
| mergeid  | ID used to connect pixels to perimeters. Matches fireid      | Numeric ID      |
| ogc_fid   |The ID used by the OGC API to sort pixels.      | Numeric ID     |
